# SPECFEM Users Workshop -- Day 1 (Oct. 5, 2022)

## Part 1C: Intro to SPECFEM3D

SPECFEM3D follows very similar practices as SPECFEM2D, so what we have learned thus far will help us understand how to run simulations in SPECFEM3D. In this notebook we will be using SPECFEM3D_Cartesian. **Any subsequent use of the term 'SPECFEM3D' will refer to 'SPECFEM3D_cartesian'** and not the other variants (SPECFEM3D_GLOBE, etc). The best reference on how to set up and use SPECFEM3D would be the user [manual](https://github.com/geodynamics/specfem3d/blob/devel/doc/USER_MANUAL/manual_SPECFEM3D_Cartesian.pdf). SPECFEM3D has already been set up in our container, and here we will focus on how to use it. In this short introduction, we will explore the SPECFEM3D repository, and then run small example problems with 3D models, starting with a homogeneous halfspace example. For simplicity we will restrict the 3D models to essentially 1D models in this workshop.

-----------

### Relevant Information

>__NOTE:__ These instructions should be run from inside the Docker container, using Jupyter Lab. The Docker container should have the adjTomo toolkit installed (SeisFlows, Pyatoa, PySEP), as well as SPECFEM2D and SPECEFM3D compiled with MPI. 

**Relevant Links:** 
- [Today's Notebook](https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/day_1c_intro_specfem3d.ipynb)
- [SPECFEM3D_Cartesian User Manual](https://github.com/geodynamics/specfem3d/blob/devel/doc/USER_MANUAL/manual_SPECFEM3D_Cartesian.pdf)
- [SPECFEM3D_Cartesian GitHub Repository](https://github.com/geodynamics/specfem3d/tree/devel)

**Jupyter Quick Tips:**

- **Run cells** one-by-one by hitting the $\blacktriangleright$ button at the top, or by hitting `Shift + Enter`
- **Run all cells** by hitting the $\blacktriangleright\blacktriangleright$ button at the top, or by running `Run -> Run All Cells`
- **Currently running cells** that are still processing will have a `[*]` symbol next to them
- **Finished cells** will have a `[1]` symbol next to them. The number inside the brackets represents what order this cell has been run in.
- Commands that start with `!` are Bash commands (i.e., commands you would run from the terminal)
- Commands that start with `%` are Jupyter Magic commands.
- To time a task, put a `%time` before the command (e.g., `%time ! ls`)

-----------

### Package exploration

The best way to get started is to look at the cloned SPECFEM3D [repository](https://github.com/geodynamics/specfem3d/tree/devel) structure.



In [ ]:
# Go to the SPECFEM3D directory
%cd /home/scoped/specfem3d

In [ ]:
# Exploring the SPECFEM3D directory
! ls

Among the many folders (and files) listed above, the most notable ones are -
- `bin/`
- `DATA/`
- `OUTPUT_FILES/`
- `EXAMPLES/`
- `src/`

Let's look at these folders one by one.


In [ ]:
! ls bin

The `bin/` folder contains binary executable files which are essentially linked compiled fortran code generated after compiling various programs available with this package. The most essential executables for a basic simulation are -
- `xmeshfem3D` - also called SPECFEM3D's internal mesher, this program discretizes the simulation domain into small elements.
- `xgenerate_databases` - this database generation program assigns model parameter values to the elements.
- `xspecfem3D` - also called the solver, this program uses the mesh and the corresponding model parameter values assigned to the mesh elements, for numerically solving the wave equation.

In [ ]:
! ls DATA

The `DATA/` folder is the input files folder and contains files and folders which describe the -
- mesh - (`meshfem3D_files/`)
- model - (`meshfem3D_files/`, `tomo_files/`)
- source - (`CMTSOLUTION`, `FORCESOLUTION`)
- station - (`STATIONS`)

In [ ]:
! ls OUTPUT_FILES

This `OUTPUT_FILES/` folder contains the output files of any SPECFEM3D job. The `DATABASES_MPI/` folder in the `OUTPUT_FILES/` folder contains the database files generated as a result of a meshing or database generation job. The database files can be very large in size. Other job files like log files for the various programs as well as the output seismograms are directly generated in the `OUTPUT_FILES/` folder.

In [ ]:
! ls src

The `src/` folder contains the source code for the various programs in SPECFEM3D. If you want to add to or modify some of SPECFEM3D's features, you need to modify one or more of the files in the src subfolders. If you think the features you added would be useful to the broader SPECFEM3D community, please considering making a pull request to the SPECFEM3D github [repository](https://github.com/geodynamics/specfem3d/tree/devel) so that it can be reviewed and integrated to the package.

In [ ]:
! ls EXAMPLES

The `EXAMPLES/` folder contains a variety of examples to provide a quick start to the user on how to use SPECFEM3D and it's various features. The README files within the example folders within `EXAMPLES/` guide the user through the steps to run each example. Some of the highly recommended examples to start with, depending on your need to run SPECFEM3D, are -

- `homogeneous halfspace/` - a simple homogeneous halfspace model based simulation
- `meshfem3D_examples/socal1D/` - a layered model based simulation
- `sensitivity_kernels_liutromp2006/` - adjoint simulations to generate the kernels in [Liu&Tromp 2006](https://pubs.geoscienceworld.org/ssa/bssa/article/96/6/2383/146674/Finite-Frequency-Kernels-Based-on-Adjoint-Methods)

>__NOTE:__ README files, with steps to run these examples, should not undermine the detailing provided in the [manual](https://github.com/geodynamics/specfem3d/blob/master/doc/USER_MANUAL) on how to run SPECFEM3D.



Here we will start with the `homogeneous halfspace/` example, and then move on to running simulations with layered models. We will also discuss some startegies to design meshes using SPECFEM3D's internal mesher.

-----------

### Examples

**Setting Up** 

It is often desireable to run SPECFEM outside of the cloned SPECFEM repository, in order to keep files and outputs manageable. SPECFEM3D only requires the following 3 directories for a successful run -
- `bin/` (with compiled executables)
- `DATA/` (with the necessary input files)
- `OUTPUT_FILES/`

In this section we will set up a separate SPECFEM3D working directory to work with.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Image

In [ ]:
# Create separate working directory for SPECFEM3D
! mkdir -p /home/scoped/work/day_1/specfem3d_workdir

In [ ]:
# Go to the SPECFEM3D working directory
%cd /home/scoped/work/day_1/specfem3d_workdir

>__NOTE:__ The following cells assume that we are in the directory `/home/scoped/work/day_1/specfem3d_workdir`, so we must evaluate the '%cd' command (if needed) to ensure that cells work as expected.

In [ ]:
# Symlink the binary files, and copy the relevant DATA/ directory
! ln -s /home/scoped/specfem3d/bin .
! mkdir -p OUTPUT_FILES/DATABASES_MPI

In [ ]:
# Look at the work directory
! ls

-----------

**1) Homogeneous halfspace** 

This example creates a homogeneous halfspace, i.e. a single volume block with a
constant elastic material property, using SPECFEM3D's internal mesher, and runs a
forward simulation.

In [ ]:
# Copy necessary input files for the homogeneous halfspace example
! cp -r /home/scoped/specfem3d/EXAMPLES/homogeneous_halfspace/DATA .
! cp -r /home/scoped/specfem3d/EXAMPLES/homogeneous_halfspace/meshfem3D_files DATA/.

# Look at the DATA directory
! ls DATA

In [ ]:
## Meshing - discretizes the simulation domain into nodes

In [ ]:
# Explore the mesh files
! ls DATA/meshfem3D_files

In [ ]:
# Check the Mesh_Par_file
! cat DATA/meshfem3D_files/Mesh_Par_file

In [ ]:
# Run mesher (xmeshfem3D)
! mpiexec -np 4 ./bin/xmeshfem3D
# Look at the generated mesh files
! ls OUTPUT_FILES/DATABASES_MPI

In [ ]:
## Database Generation - assigns model values to the mesh nodes

In [ ]:
# Run database generator (xgenerate_databases)
! mpiexec -np 4 ./bin/xgenerate_databases
# Explore the generated database files
! ls OUTPUT_FILES/DATABASES_MPI

In [ ]:
## View the meshing and the Vp model
# The following image was generated using Paraview
! cp /home/scoped/adjdocs/workshops/2022-10-05_specfem_users/additional_material/day_1c/figures/mesh/example_A.png .
Image("example_A.png")

In [ ]:
## Simulation - solves for the output quantities (displacement, velocity, acceleration, etc) at the nodes

In [ ]:
# Explore the solver files

In [ ]:
# Check the source definition
! cat DATA/CMTSOLUTION

Read more about the CMTSOLUTION files [here](https://www.sciencedirect.com/science/article/pii/S0031920112000696?via%3Dihub).

In [ ]:
# Check the stations file
! cat DATA/STATIONS

In [ ]:
# View the source station geometry
# The following image was pre-generated using Paraview
! cp /home/scoped/adjdocs/workshops/2022-10-05_specfem_users/additional_material/day_1c/figures/source_station_geometry/sr.png .
Image("sr.png")

In [ ]:
# Check the Par_file
! head -30 DATA/Par_file

In [ ]:
! head -80 DATA/Par_file | tail -n 22

In [ ]:
# Run the solver (xspecfem3D)
%time ! mpiexec -np 4 ./bin/xspecfem3D

>__NOTE:__ solver runs should take ~5 minutes or less on a recent (as on 2022) laptop

In [ ]:
# Look at the simulation output files
! ls OUTPUT_FILES/

In [ ]:
# Explore the seismograms
! head -25 OUTPUT_FILES/DB.X50.BXZ.semd 

In [ ]:
# Plot the seismograms

X20_Z = np.genfromtxt("OUTPUT_FILES/DB.X20.BXZ.semd", dtype=None, names=("time","BXZ"))
X30_Z = np.genfromtxt("OUTPUT_FILES/DB.X30.BXZ.semd", dtype=None, names=("time","BXZ"))
X40_Z = np.genfromtxt("OUTPUT_FILES/DB.X40.BXZ.semd", dtype=None, names=("time","BXZ"))
X50_Z = np.genfromtxt("OUTPUT_FILES/DB.X50.BXZ.semd", dtype=None, names=("time","BXZ"))

t = X20_Z["time"]

plt.title("Seismograms (Z - component)")
plt.xlabel("---- time -->")
plt.ylabel("---- displacement -->")

plt.plot(t,X20_Z["BXZ"],label="X20")
plt.plot(t,X30_Z["BXZ"],label="X30")
plt.plot(t,X40_Z["BXZ"],label="X40")
plt.plot(t,X50_Z["BXZ"],label="X50")

plt.legend(title="Station")
plt.savefig("seis.png")

In [ ]:
# Archive the output files, and the mesh files
! mv sr.png OUTPUT_FILES/.
! mv example_A.png OUTPUT_FILES/mesh_examples_A.png
! mv seis.png OUTPUT_FILES/.
! mv OUTPUT_FILES OUTPUT_FILES_example_A
! mv DATA/meshfem3D_files DATA/meshfem3D_files_example_A

-----------

**2) Two layered model**

This example is built upon the homogeneous halfspace model of the previous example, with the bottom three-fourths of the domain assigned higher velocities and density.

In [ ]:
# Setup
! cp -r /home/scoped/adjdocs/workshops/2022-10-05_specfem_users/additional_material/day_1c/meshfem3D_files/example_B DATA/meshfem3D_files
! mkdir -p OUTPUT_FILES/DATABASES_MPI

In [ ]:
# Check the difference
! ls DATA/meshfem3D_files
! diff DATA/meshfem3D_files_example_A/Mesh_Par_file DATA/meshfem3D_files/Mesh_Par_file 

In [ ]:
# Run mesher (xmeshfem3D)
! mpiexec -np 4 ./bin/xmeshfem3D

In [ ]:
# Run database generator (xgenerate_databases)
! mpiexec -np 4 ./bin/xgenerate_databases

In [ ]:
## View the meshing and the Vp model
# The following image was pre-generated using Paraview
! cp /home/scoped/adjdocs/workshops/2022-10-05_specfem_users/additional_material/day_1c/figures/mesh/example_B.png .
Image("example_B.png")

In [ ]:
# Run the solver (xspecfem3D)
%time ! mpiexec -np 4 ./bin/xspecfem3D

>__NOTE:__ solver runs should take ~5 minutes or less on a recent (as on 2022) laptop

In [ ]:
# Plot the output seismograms

X20_Z = np.genfromtxt("OUTPUT_FILES/DB.X20.BXZ.semd", dtype=None, names=("time","BXZ"))
X30_Z = np.genfromtxt("OUTPUT_FILES/DB.X30.BXZ.semd", dtype=None, names=("time","BXZ"))
X40_Z = np.genfromtxt("OUTPUT_FILES/DB.X40.BXZ.semd", dtype=None, names=("time","BXZ"))
X50_Z = np.genfromtxt("OUTPUT_FILES/DB.X50.BXZ.semd", dtype=None, names=("time","BXZ"))

t = X20_Z["time"]

plt.title("Seismograms (Z - component)")
plt.xlabel("---- time -->")
plt.ylabel("---- displacement -->")

plt.plot(t,X20_Z["BXZ"],label="X20")
plt.plot(t,X30_Z["BXZ"],label="X30")
plt.plot(t,X40_Z["BXZ"],label="X40")
plt.plot(t,X50_Z["BXZ"],label="X50")

plt.legend(title="Station")
plt.savefig("seis.png")

In [ ]:
# Archive the output files, and the mesh files
! mv example_B.png OUTPUT_FILES/mesh_examples_B.png
! mv seis.png OUTPUT_FILES/.
! mv OUTPUT_FILES OUTPUT_FILES_example_B
! mv DATA/meshfem3D_files DATA/meshfem3D_files_example_B

-----------

**3) Two layered model with modified mesh**

This example is built upon the two layered model used in the previous example, with larger elements (cubes with side lengths doubled) for the higher velocity layer.

In [ ]:
# Setup
! cp -r /home/scoped/adjdocs/workshops/2022-10-05_specfem_users/additional_material/day_1c/meshfem3D_files/example_C DATA/meshfem3D_files
! mkdir -p OUTPUT_FILES/DATABASES_MPI

In [ ]:
# Check the difference
! ls DATA/meshfem3D_files
! diff DATA/meshfem3D_files_example_B/Mesh_Par_file DATA/meshfem3D_files/Mesh_Par_file 

In [ ]:
# Run mesher (xmeshfem3D)
! mpiexec -np 4 ./bin/xmeshfem3D

In [ ]:
# Run database generator (xgenerate_databases)
! mpiexec -np 4 ./bin/xgenerate_databases

In [ ]:
## View the meshing and the Vp model
# The following image was pre-generated using Paraview
! cp /home/scoped/adjdocs/workshops/2022-10-05_specfem_users/additional_material/day_1c/figures/mesh/example_C.png .
Image("example_C.png")

In [ ]:
# Run the solver (xspecfem3D)
%time ! mpiexec -np 4 ./bin/xspecfem3D

>__NOTE:__ solver runs should take ~5 minutes or less on a recent (as on 2022) laptop

In [ ]:
# Plot the output seismograms

X20_Z = np.genfromtxt("OUTPUT_FILES/DB.X20.BXZ.semd", dtype=None, names=("time","BXZ"))
X30_Z = np.genfromtxt("OUTPUT_FILES/DB.X30.BXZ.semd", dtype=None, names=("time","BXZ"))
X40_Z = np.genfromtxt("OUTPUT_FILES/DB.X40.BXZ.semd", dtype=None, names=("time","BXZ"))
X50_Z = np.genfromtxt("OUTPUT_FILES/DB.X50.BXZ.semd", dtype=None, names=("time","BXZ"))

t = X20_Z["time"]

plt.title("Seismograms (Z - component)")
plt.xlabel("---- time -->")
plt.ylabel("---- displacement -->")

plt.plot(t,X20_Z["BXZ"],label="X20")
plt.plot(t,X30_Z["BXZ"],label="X30")
plt.plot(t,X40_Z["BXZ"],label="X40")
plt.plot(t,X50_Z["BXZ"],label="X50")

plt.legend(title="Station")
plt.savefig("seis.png")

So by increasing the element size for the higher velocity layer, we are able to speed up the simulation greatly without loosing out on the accuracy of results. A note of caution is that changing mesh dimensions with change in velocity can be challenging at times and needs a good amount of care to ensure the accuracy of results. In some cases the numerical solver runs successfully, but the results maybe completely incorrect or maybe overshadowed by artifacts.

In [ ]:
# Archive the output files, and the mesh files
! mv example_C.png OUTPUT_FILES/mesh_examples_C.png
! mv seis.png OUTPUT_FILES/.
! mv OUTPUT_FILES OUTPUT_FILES_example_C
! mv DATA/meshfem3D_files DATA/meshfem3D_files_example_C

-----------

**Congratulations! You have reached your destination.**